Hugging face transfomers ,datsets,evaluate like BLEU

In [1]:
# Uninstall W&B to avoid errors
!pip uninstall -y wandb

# Install Hugging Face libraries
!pip install -U transformers datasets evaluate -q

Found existing installation: wandb 0.20.1
Uninstalling wandb-0.20.1:
  Successfully uninstalled wandb-0.20.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0

In [2]:
import os

# Disable wandb before it's ever used
os.environ["WANDB_DISABLED"] = "true"

# Optional but good practice
os.environ["WANDB_MODE"] = "offline"


In [3]:
import transformers
print(transformers.__version__)

4.52.4


In [5]:
import pandas as pd

# Load your CSV files
train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")

# Combine Title and Description into a single text column
train_df["text"] = train_df["Title"] + " " + train_df["Description"]
test_df["text"] = test_df["Title"] + " " + test_df["Description"]

# Convert class index from 1-4 to 0-3
train_df["label"] = train_df["Class Index"] - 1
test_df["label"] = test_df["Class Index"] - 1

# Keep only necessary columns
train_df = train_df[["text", "label"]]
test_df = test_df[["text", "label"]]


In [6]:
train_df.sample(5)

,text,label
44906,Thailand to Change Farming Ways to End Bird Fl...,0
112687,Bristol-Myers to Seek Over-The-Counter OK Bris...,2
108302,Forecaster Sees Fewer Atlantic Hurricanes in 2...,2
72988,U.S. Expats Jump on Planes to Vote in Home Sta...,0
18775,Govt sticks to army law despite protests (Reut...,0


In [7]:
from datasets import Dataset

# Convert to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Combine into a DatasetDict
from datasets import DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})


In [8]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 120000
})

In [9]:
test_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 7600
})

In [10]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)

encoded_dataset = dataset.map(tokenize, batched=True)
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [11]:
encoded_dataset["train"][0]

{'label': tensor(2),
 'input_ids': tensor([  101,  2813,  2358,  1012,  6468, 15020,  2067,  2046,  1996,  2304,
          1006, 26665,  1007, 26665,  1011,  2460,  1011, 19041,  1010,  2813,
          2395,  1005,  1055,  1040, 11101,  2989,  1032,  2316,  1997, 11087,
          1011, 22330,  8713,  2015,  1010,  2024,  3773,  2665,  2153,  1012,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,   

In [12]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import evaluate
import numpy as np

# Load model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)

# Define metrics
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

# Training setup
# Ensure transformers is up-to-date before this
training_args = TrainingArguments(
    output_dir="./results",
    # 'evaluation_strategy' was renamed to 'eval_strategy' in newer versions
    eval_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=50,
    weight_decay=0.01
)

# Train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    compute_metrics=compute_metrics
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [13]:
for param in model.distilbert.parameters():   #transfer learning so freezing the layers of distillbert
    param.requires_grad = False

In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.292100,0.293752,0.900000
2,0.282400,0.279110,0.900921
3,0.238300,0.275148,0.902763


TrainOutput(global_step=22500, training_loss=0.3047311294555664, metrics={'train_runtime': 3342.4365, 'train_samples_per_second': 107.706, 'train_steps_per_second': 6.732, 'total_flos': 2.384498221056e+16, 'train_loss': 0.3047311294555664, 'epoch': 3.0})

In [15]:
trainer.evaluate()

{'eval_loss': 0.2751477062702179,
 'eval_accuracy': 0.9027631578947368,
 'eval_runtime': 53.715,
 'eval_samples_per_second': 141.488,
 'eval_steps_per_second': 8.843,
 'epoch': 3.0}

In [22]:
import torch

# Set the device (automatically handles GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move model to the right device

# Updated predict function
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to same device
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_class = outputs.logits.argmax(dim=1).item()
    return predicted_class


text = "Virat kohli won the ipl trophy"
print("Predicted class index:", predict(text))



Predicted class index: 1


In [23]:
label_map = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}
print("Predicted Label:", label_map[predict(text)])


Predicted Label: Sports
